In [1]:
import os.path as osp
import os
from concurrent.futures import ProcessPoolExecutor

import numpy as np

from tqdm import tqdm_notebook as tqdm
from msd import MSD_MELSPEC as MSD

import features

%load_ext autoreload
%autoreload 2

In [2]:
dataset = MSD(phase='all')

In [3]:
def process_single(paths):
    input_path, output_path = paths
    if osp.exists(output_path):
        return
    try:
        x = features.get_audio(input_path)
    except Exception:
        print(input_path)
        return
    try:
        with open(output_path, 'wb') as f:
            np.save(f, x)
    except Exception:
        if osp.exists(output_path):
            os.remove(output_path)
    
with ProcessPoolExecutor() as executor:
    paths = [(osp.join('/share/MSD/{}'.format(x)),
              osp.join('/share/MSD/precomputed/vanilla/{}.npy'.format(x))) for x in dataset.X]
    total_chunks = 10000
    for processed in tqdm(executor.map(process_single, paths, chunksize=len(paths)//total_chunks),
                          total=len(paths)):
        pass